In [1]:
import numpy as np
from scipy import interpolate
from netCDF4 import Dataset
import xarray
import pdb

In [2]:
def select(dt, lon1, lon2, lat1, lat2):
    #经纬度坐标全部化为整数
    lon1, lon2, lat1, lat2 = [int(i)-0.125 for i in [lon1, lon2, lat1, lat2]] #Fix me: hard code
    
    roi_lon = np.arange(lon1, lon2, 0.25)
    roi_lat = np.arange(lat1, lat2, 0.25)
    roi_dt = dt.sel(lon=roi_lon, lat=roi_lat) 
    return roi_dt

def interp(dt, step_lon, step_lat=None):
    
    minvalue = np.nanmin(dt.values)
    maxvalue = np.nanmax(dt.values)
    
    
    roi_lon, roi_lat = dt.lon, dt.lat
    # roi_lon, roi_lat = roi_lon.flatten(), roi_lat.flatten()
    roi_adt = dt.values
    roi_adt[np.isnan(roi_adt)] = -2**31+1

    if step_lat == None:
        step_lat = step_lon


    #生成插值器
    interpolator_cubic = interpolate.interp2d(roi_lon, roi_lat, roi_adt, kind='cubic')
    interpolator_linear = interpolate.interp2d(roi_lon, roi_lat, roi_adt, kind='linear')
    print('done')

    #插值,生成插值后数据
    X1 = np.arange(roi_lon[0], roi_lon[-1], step_lon)
    Y1 = np.arange(roi_lat[0], roi_lat[-1], step_lat)
    
    
    #Fix me:High cost
    data_cubic = interpolator_cubic(X1, Y1)
    data_linear = interpolator_linear(X1, Y1)
    mask_linear=(np.isnan(data_linear) | (data_linear<minvalue) | (data_linear>maxvalue) )
    data_linear[mask_linear]=np.nan
    mask_cubic=(np.isnan(data_cubic) | (data_cubic<minvalue) | (data_cubic>maxvalue) )
    data_cubic[mask_cubic]=np.nan
    
    residual = np.abs(data_cubic-data_linear)
    Q1 = np.nanquantile(residual, 0.75)
    Q3 = np.nanquantile(residual, 0.25)
    IQR = Q1-Q3
    upbound = Q1+1.5*IQR
    lowbound = Q3-1.5*IQR
    mask_cubic = np.isnan(residual) | (residual>upbound) | (residual<lowbound)
    
    data = np.where(mask_cubic, data_linear, data_cubic)
    data[mask_linear]=np.nan
    data = xarray.DataArray(data, dims=('lat', 'lon'), coords=dict(lon=X1, lat=Y1))
    
    return data


def write(data, path, roi_lon, roi_lat):
    with Dataset(path, 'w', format="NETCDF4") as dataset:
        dataset.createDimension('longitude', data.shape[1])
        dataset.createDimension('latitude', data.shape[0])

        lon = dataset.createVariable('longitude', 'f4', ('longitude',))
        lat = dataset.createVariable('latitude', 'f4', ('latitude',))
        adt_ = dataset.createVariable('adt', 'f4', ('latitude', 'longitude'))

        lon[:] = roi_lon
        lat[:] = roi_lat
        adt_.units='m'
        adt_[:] = data

    print('closed')
    
def xy2geo(roi_lon, roi_lat, candidates):    
    points = np.array([roi_lon[candidates[..., 0],candidates[..., 1]],roi_lat[candidates[..., 0],candidates[..., 1]] ]).T
    return points